<a href="https://colab.research.google.com/github/vvampp/PLN-ESCOMCoursePractices/blob/main/Proyecto/Final_Proyect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install torch
!pip install rouge_score
!pip install datasets

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=42264b063a6f67d5faa67898eabec64f4f16e0c6854392c2b7d9f1c23d3de5ed
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [10]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
from rouge_score import rouge_scorer

# 1. Cargar modelo y tokenizador
modelo = "mrm8488/bert2bert_shared-spanish-finetuned-summarization"
tokenizer = AutoTokenizer.from_pretrained(modelo)
model = AutoModelForSeq2SeqLM.from_pretrained(modelo)

# 2. Cargar el dataset XLSum para textos en español
dataset = load_dataset("csebuetnlp/xlsum", "spanish")

# Seleccionar un artículo para la prueba
ejemplo = dataset["train"][0]
articulo = ejemplo["text"]
resumen_referencia = ejemplo["summary"]

# 3. Función para generar un resumen
def generate_summary(article):
    inputs = tokenizer(article, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs["input_ids"], max_length=128, min_length=40, length_penalty=2.0, num_beams=4)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Generar el resumen
generated_summary = generate_summary(articulo)

# Obtener variable
articulo = articulo[:500]




Config of the encoder: <class 'transformers.models.bert.modeling_bert.BertModel'> is overwritten by shared encoder config: BertConfig {
  "_name_or_path": "dccuchile/bert-base-spanish-wwm-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.47.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31002
}

Config of the decoder: <class 'transformers.models.bert.modeling_bert.BertLMHeadModel'> is overwritten by shared decoder config: BertConfig {
  "_name_or_path": "dccuchile/bert-base-span

In [11]:
# Resultados
print("Artículo original:")
print(articulo)
print("\nResumen de referencia:")
print(resumen_referencia)
print("\nResumen generado:")
print(generated_summary)

Artículo original:
De no recibir más dinero, las raciones de la ONU en Siria se terminarán en dos meses. En un informe al Consejo de Seguridad de la ONU, Amos dijo que las raciones del PMA destinadas a los 4.000.000 de sirios ya han sido recortadas para poder llegar a la mayor cantidad de personas posible. Amos también hizo un llamado a juntar suministros para proteger a los sirios del frío, en miras al próximo invierno.

Resumen de referencia:
La directora de ayuda humanitaria de Naciones Unidas, Valerie Amos, advirtió que de no invertir más dinero, el Programa Mundial de Alimentos tendrá que detener sus operaciones en Siria en dos meses.

Resumen generado:
Amos dijo que las raciones del PMA destinadas a los 4. 000. 000 de sirios ya han sido recortadas para poder llegar a la mayor cantidad de personas posible. Amos también hizo un llamamiento a juntar suministros para proteger a los sirios del frío, en miras al próximo invierno


In [12]:
# 4. Evaluar con ROUGE
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(resumen_referencia, generated_summary)

print("\nPuntuaciones ROUGE:")
for key, value in scores.items():
    print(f"{key}: Recall: {value.recall:.2f}, Precision: {value.precision:.2f}, F1: {value.fmeasure:.2f}")


Puntuaciones ROUGE:
rouge1: Recall: 0.18, Precision: 0.12, F1: 0.14
rouge2: Recall: 0.00, Precision: 0.00, F1: 0.00
rougeL: Recall: 0.15, Precision: 0.10, F1: 0.12
